In [10]:
import sys
sys.path.append('../../')

from bokehComponents.Dashboard import Dashboard
from bokehComponents.BokehComponents import BokehControl,BufferedQueryInterface,BokehTimeseriesGraphic

from bokeh.io import output_notebook,output_file
from bokeh.application.handlers import FunctionHandler
from bokeh.application import Application
from bokeh.models.widgets import Panel, Tabs #, DataTable, DateFormatter, TableColumn, Tabs, 
from bokeh.models import ColumnDataSource
from bokeh.layouts import layout
from bokeh.plotting import figure,show,output_file
from bokeh.models import Div

from datetime import datetime,timedelta
import pandas_datareader as pdr


In [11]:
from bokeh.plotting import figure, output_file, show



# 3 Custom Data Sources
In general, the major activity behind the scenes with bokeh is the management of data. It's not the most fun subject. In this example I show how to create custom and versitile data sources that can be used across a variety of bokehComponents.

### Status: In Development
Note, the library is in active development.

## 3.1 Creating a complex query BufferedQueryInterface
With all Bokeh components, it can be useful to abstrat the data interface from the rest of the application. Previously we created a pretty simple query interface. In this interface, we will update it to handle periodic updates. This idea of periodic updates will be used in the future by plots, tables, and other systems. 
First, extend the BufferedQueryInterface():
##### Extend the BufferedQueryInterface

In [12]:
class ExampleComplexQuery(BufferedQueryInterface):
        pass


##### Create a custom append action
This is an example method that appends data. There is nothing special about the naming of this method.

In [13]:

def appendNextAction(self,indicesIn= None):
    #
    ## First, you are given selected data indices. Sometimes this may be null. in this action we will ignore this index.
    
    assert indicesIn or not indiciesIn # do something with indices if you want
    
    #
    ## Second, do something with self.data

    next_day = self.current_date+ timedelta(days=3)
    data=pdr.get_data_yahoo('AAPL',self.current_date,next_day)
    self.data['open_list'].append(data['Open'][0])
    self.data['close_list'].append(data['Close'][0])
    self.data['date_time'].append(next_day)
    assert len(self.data['open_list']) == len(self.data['close_list'])
    assert len(self.data['close_list']) == len(self.data['date_time'])

    self.current_date = next_day   
    


##### create a custom delete action

In [14]:
def deleteAction(self,indicesIn= None):
    #
    ## Remove all the indices, from largest to smallest
    tables = ['open_list','close_list','date_time']
    indicesIn.sort(reverse=True)
    for i in indicesIn:
        for tableName in tables:
            self.data[tableName].pop(i)


##### Make self.data, regiseter actions.
Second, implement the load_data_buffer method. This method is required to create a self.data dictionary. If this dictionary is not created then terrible things will happen. Make self.data.

In [15]:
def load_data_buffer(self):
    try:
        if (self.init):
            return
    except:
        pass
    self.init = 1
    start_init=self._settings['start_date']
    end_init=self._settings['end_date']     
    self.current_date = end_init 
    # Grab data
    start=start_init
    end=end_init 

    data=pdr.get_data_yahoo('AAPL',start,end)
    data_sorted=data.sort_index(axis=0,ascending=True)
    date_list=list(data_sorted.index)

    # Save the data elements
    open_list=list(data_sorted['Open'])
    close_list=list(data_sorted['Close'])
    date_time=[datetime.strptime(str(d),'%Y-%m-%d %H:%M:%S').date() for d in date_list]

    #
    ## First, create the data
    self.data = {
        'open_list':open_list,
        'close_list':close_list,
        'date_time':date_time,
        }
    
    #
    ## Second, register the append action so it can be used.
    self.registerAction('append',appendNextAction)
    self.registerAction('delete',deleteAction)
    
ExampleComplexQuery.load_data_buffer = load_data_buffer

# 3.2 Using the Query Object

### 3.2.1 Query Some Data

In [16]:
import pandas as pd
query = ExampleComplexQuery({'start_date':datetime(2017,4,1),'end_date':datetime(2017,4,10)})
data = query.QueryData()
df = pd.DataFrame(data)
display(df)

,open_list,close_list,date_time
0,143.710007,143.699997,2017-04-03
1,143.250000,144.770004,2017-04-04
2,144.220001,144.020004,2017-04-05
3,144.289993,143.660004,2017-04-06
4,143.729996,143.339996,2017-04-07
5,143.600006,143.169998,2017-04-10


### 3.2.1 Use Custom Delete action 

In [26]:
query.DoAction(action_id = 'delete',query=None,indicesIn =[0])
data = query.QueryData()
df = pd.DataFrame(data)
display(df)

,open_list,close_list,date_time
0,143.600006,143.169998,2017-04-10
1,143.600006,143.169998,2017-04-13
2,141.910004,141.050003,2017-04-16
3,141.479996,141.830002,2017-04-19
4,141.880005,140.679993,2017-04-22
5,143.500000,143.639999,2017-04-25


### 3.2.1 Use Append action 

In [23]:
query.DoAction(action_id = 'append')
data = query.QueryData()
df = pd.DataFrame(data)
display(df)

,open_list,close_list,date_time
0,144.220001,144.020004,2017-04-05
1,144.289993,143.660004,2017-04-06
2,143.729996,143.339996,2017-04-07
3,143.600006,143.169998,2017-04-10
4,143.600006,143.169998,2017-04-13
5,141.910004,141.050003,2017-04-16
6,141.479996,141.830002,2017-04-19
7,141.880005,140.679993,2017-04-22
8,143.500000,143.639999,2017-04-25


### Using the default BokehTimeseriesGraphic
It is possible use the graphic timeseries out of the box

In [29]:
class GraphDashboard(Dashboard):
    def getLayout(self):
        settings = {'title':'Analytics Plot',
                          'width':600,
                          'height':300,
                          'query':ExampleComplexQuery({'start_date':datetime(2017,4,26),
                                               'end_date':datetime(2017,5,26)})}

        settings['data_defs'] = [
                    {'y':'open_list','x':'date_time','key':'open','label':'Open','color':'green' },
                    {'y':'close_list','x':'date_time','key':'close','label':'Close','color':'yellow' }                    
                    ]
        
        self.g = BokehTimeseriesGraphic(settings)
        l = layout([self.g.getBokehComponent()],sizing_mode='fixed')         
        return l    
    
d = GraphDashboard()
d.showDashboard()

Loading BokehJS ...